In [1]:
import sys
sys.path.insert(0,'/home/caldaz/module/pyGSM')
from molecule import Molecule
from pes import PES
from avg_pes import Avg_PES
import numpy as np
from nifty import pvec1d,pmat2d
import matplotlib
import matplotlib.pyplot as plt
from pytc import *
import manage_xyz
from rhf_lot import *
from psiw import *
from nifty import getAllCoords,getAtomicSymbols,click,printcool
import pybel as pb
%matplotlib inline

In [2]:
printcool("Build resources")
resources = ls.ResourceList.build()
printcool('{}'.format(resources))

#========================================================#
#|                   Build resources                    |#
#========================================================#
#==============================================================#
#|                       ResourceList:                        |#
#|                       CPU threads:   1                     |#
#|                       GPU contexts:  1                     |#
#|                     GPU context details:                   |#
#|    N  ID                 Name  CC  CPU Buffer  GPU Buffer  |#
#|    0   0    GeForce GTX TITAN 3.5        1024        1024  |#
#|                                                            |#
#==============================================================#


'----------------------------------------------------------------\n'

In [3]:
printcool("build the Lightspeed (pyTC) objecs")

filepath='../data/twisted_fluoroethene.xyz'
nocc=11

molecule = ls.Molecule.from_xyz_file(filepath)
geom = geometry.Geometry.build(
    resources=resources,
    molecule=molecule,
    basisname='6-31gs',
    )
printcool('{}'.format(geom))

ref = RHF.from_options(
     geometry= geom, 
     g_convergence=1.0E-6,
     fomo=True,
     fomo_method='gaussian',
     fomo_temp=0.3,
     fomo_nocc=nocc,
     fomo_nact=2,
     print_level=1,
    )
ref.compute_energy()
casci = CASCI.from_options(
    reference=ref,
    nocc=nocc,
    nact=2,
    nalpha=1,
    nbeta=1,
    S_inds=[0],
    S_nstates=[2],
    print_level=1,
    )
casci.compute_energy()
psiw = CASCI_LOT.from_options(
    casci=casci,
    rhf_guess=True,
    rhf_mom=True,
    orbital_coincidence='core',
    state_coincidence='full',
    )

#========================================================#
#|          build the Lightspeed (pyTC) objecs          |#
#========================================================#
#========================================================#
#|                      Geometry:                       |#
#|                      QMMM = False                    |#
#|                      -D3  = False                    |#
#|                      ECP  = False                    |#
#|                                                      |#
#|            Molecule: twisted_fluoroethene            |#
#|               Natom        =           6             |#
#|               Charge       =       0.000             |#
#|               Multiplicity =       1.000             |#
#|                                                      |#
#|                    Basis: 6-31gs                     |#
#|                    nao     =    51                   |#
#|                    ncart   =    51                   

In [4]:
printcool("Build the pyGSM Level of Theory object (LOT)")
lot=PyTC.from_options(states=[(1,0),(1,1)],job_data={'psiw':psiw},fnm=filepath)

#========================================================#
#|     Build the pyGSM Level of Theory object (LOT)     |#
#========================================================#
 initializing LOT from file


In [5]:
printcool("Build the pyGSM Potential Energy Surface Object (PES)")
pes = PES.from_options(lot=lot,ad_idx=0,multiplicity=1)


#===========================================================#
#|  Build the pyGSM Potential Energy Surface Object (PES)  |#
#===========================================================#


In [6]:
printcool("Build the pyGSM Molecule object \n with Translation and Rotation Internal Coordinates (TRIC)")
M = Molecule.from_options(fnm=filepath,PES=pes,coordinate_type="TRIC")

#================================================================#
#|               Build the pyGSM Molecule object                |#
#|   with Translation and Rotation Internal Coordinates (TRIC)  |#
#================================================================#
 reading cartesian coordinates from file
 initializing LOT from file
 making primitives from options!
 making primitive Hessian
 forming Hessian in basis


In [7]:
from lbfgs import lbfgs
from _linesearch import backtrack
lb = lbfgs.from_options(DMAX=0.25) #Linesearch=backtrack

In [8]:
geoms,energy = lb.optimize(M,M.energy,'UNCONSTRAINED',15)

 initial E 0.0000
 On opt step 1 
 actual_step= 0.68
 reducing step, new step = 0.25
 Linesearch
 Done linesearch
 dEstep=-43.2588
 Opt step: 1 E: -43.2588 gradrms: 0.16090 ss: 0.250 DMAX: 0.250
 On opt step 2 
 actual_step= 0.09
 Linesearch
 Done linesearch
 dEstep=-3.8355
 Opt step: 2 E: -47.0943 gradrms: 0.02128 ss: 0.092 DMAX: 0.250
 On opt step 3 
 actual_step= 0.28
 reducing step, new step = 0.25
 Linesearch
 Done linesearch
 dEstep=-8.8025
 Opt step: 3 E: -55.8967 gradrms: 0.01850 ss: 0.250 DMAX: 0.250
 On opt step 4 
 actual_step= 0.76
 reducing step, new step = 0.25
 Linesearch
 Done linesearch
 dEstep=-7.5862
 Opt step: 4 E: -63.4829 gradrms: 0.02353 ss: 0.250 DMAX: 0.250
 On opt step 5 
 actual_step= 0.79
 reducing step, new step = 0.25
 Linesearch
 Done linesearch
 dEstep=-6.4240
 Opt step: 5 E: -69.9069 gradrms: 0.02382 ss: 0.250 DMAX: 0.250
 On opt step 6 
 actual_step= 0.70
 reducing step, new step = 0.25
 Linesearch
 Done linesearch
 dEstep=-5.3948
 Opt step: 6 E: -75.3

In [9]:
manage_xyz.write_xyzs_w_comments('lbfgs_DLC.xyz',geoms,energy,scale=1.)

In [18]:
np.dot(M.coord_basis,M.coord_basis.T)

array([[ 0.9991,  0.    , -0.    ,  0.0039, -0.002 , -0.    , -0.    ,
        -0.0083, -0.0041, -0.    ,  0.0254, -0.0053,  0.0053, -0.0053,
         0.0053, -0.    , -0.    ,  0.    , -0.    ,  0.    , -0.    ],
       [ 0.    ,  1.    , -0.    , -0.    , -0.    , -0.    ,  0.    ,
         0.    , -0.    , -0.    ,  0.    , -0.    , -0.    ,  0.    ,
         0.    , -0.    ,  0.    , -0.    , -0.    , -0.    ,  0.    ],
       [-0.    , -0.    ,  1.    ,  0.    , -0.    , -0.    , -0.    ,
        -0.    ,  0.    ,  0.    , -0.    ,  0.    ,  0.    , -0.    ,
        -0.    ,  0.    , -0.    ,  0.    ,  0.    ,  0.    , -0.    ],
       [ 0.0039, -0.    ,  0.    ,  0.982 ,  0.0093,  0.    , -0.    ,
         0.0378,  0.0187, -0.    , -0.116 ,  0.0242, -0.0242,  0.0242,
        -0.0242,  0.    ,  0.    , -0.    ,  0.    , -0.    ,  0.    ],
       [-0.002 , -0.    , -0.    ,  0.0093,  0.9952,  0.    , -0.    ,
        -0.0194, -0.0096,  0.    ,  0.0597, -0.0124,  0.0124, -0.0124,
  

In [17]:
ans = np.dot(M.coord_basis.T,M.coord_basis)


[[ 1. -0.  0. -0.  0. -0.  0.  0.  0.  0. -0. -0.  0. -0. -0. -0.  0. -0.]
 [-0.  1. -0. -0.  0. -0.  0.  0.  0.  0. -0. -0. -0.  0. -0.  0. -0. -0.]
 [ 0. -0.  1.  0. -0. -0.  0. -0.  0.  0. -0. -0.  0. -0.  0.  0.  0.  0.]
 [-0. -0.  0.  1.  0.  0. -0. -0. -0.  0.  0.  0.  0. -0. -0. -0.  0.  0.]
 [ 0.  0. -0.  0.  1.  0. -0.  0.  0.  0.  0.  0.  0. -0. -0. -0.  0.  0.]
 [-0. -0. -0.  0.  0.  1.  0. -0.  0.  0.  0. -0.  0. -0. -0. -0. -0. -0.]
 [ 0.  0.  0. -0. -0.  0.  1.  0. -0.  0.  0. -0. -0.  0. -0.  0.  0. -0.]
 [ 0.  0. -0. -0.  0. -0.  0.  1. -0. -0. -0. -0. -0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. -0.  0.  0. -0. -0.  1.  0.  0.  0.  0. -0.  0. -0. -0. -0.]
 [ 0.  0.  0.  0.  0.  0.  0. -0.  0.  1. -0. -0.  0. -0.  0.  0. -0. -0.]
 [-0. -0. -0.  0.  0.  0.  0. -0.  0. -0.  1.  0.  0. -0. -0.  0. -0.  0.]
 [-0. -0. -0.  0.  0. -0. -0. -0.  0. -0.  0.  1. -0. -0.  0. -0.  0.  0.]
 [ 0. -0.  0.  0.  0.  0. -0. -0.  0.  0.  0. -0.  1.  0. -0.  0. -0.  0.]
 [-0.  0. -0. -0. -0. -0.